<a href="https://colab.research.google.com/github/lechurnair/TempQA/blob/main/validity_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import spacy
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Lexical Approach

In [2]:
counter=0
c=0

def check_temporal_connotation(predicted_question, context):
    global counter
    global c
    c+=1
    doc = nlp(predicted_question)

    context_doc = nlp(context)
    words = []
    for token in context_doc:
        if token.pos_ in ["NOUN", "PROPN", "VERB"] and token.dep_ != "aux":
            words.append(token.lemma_.lower())

    flag = False
    for word in words:
        if word and (word in [token.lemma_.lower() for token in doc]):
            counter += 1
            flag = True
            break

    if not flag:
        return False

    for token in doc:
        if token.dep_ == "advmod":
            if token.text.lower() in ["how", "when", "how often", "how long", "how frequent"]:
                return True
        elif token.dep_ == "aux" and token.text.lower() == "when":
            return True
        elif token.dep_ == "prep" and token.text.lower() in ["during", "at"]:
            return True
        elif token.text.lower() in ["after", "before", "tomorrow", "yesterday"]:
            return True
        elif token.dep_ == "aux" and (token.text.lower() == "will" or token.text.lower() == "is" or token.text.lower() == "are"):
            return True
        elif token.dep_ == "ROOT" and (token.text.lower() == "is" or token.text.lower() == "are"):
            return True
    return False

#Use the required file. Each of the following files have questions generated from some dataset

# data = pd.read_excel("/content/drive/MyDrive/generated_questions_sampling.xlsx")
# data = pd.read_csv("/content/drive/MyDrive/Aru/question_cnnDailyMail.csv")
# data = pd.read_csv("/content/drive/MyDrive/Aru/question_Samsum_test.csv")
data = pd.read_csv("/content/drive/MyDrive/question_RealNews.csv")

filtered_data = data[data.apply(lambda row: check_temporal_connotation(row["predicted_question"], row["context"]), axis=1)]

Semantic Similarity

In [5]:
counter=0
c=0
def check_temporal_connotation(predicted_question, context):
    global counter
    global c
    c += 1
    doc = nlp(predicted_question)
    context_doc = nlp(context)
    noun_phrases = [chunk.text.lower() for chunk in context_doc.noun_chunks]

    words = [token.text.lower() for token in doc if token.pos_ in ["NOUN", "PROPN", "VERB"]]
    phrases = [' '.join(words[i:i+2]) for i in range(len(words) - 1)]

    flag = False
    for phrase in phrases:
        phrase_doc = nlp(phrase)
        similarity_scores = [phrase_doc.similarity(nlp(np)) for np in noun_phrases]
        max_score = max(similarity_scores, default=0)
        #Can be adjusted
        if max_score > 0.5:
            counter += 1
            flag = True
            break

    if not flag:
        return False

    for token in doc:
        if token.dep_ == "advmod":
            if token.text.lower() in ["how", "when", "how often", "how long", "how frequent"]:
                return True
        elif token.dep_ == "aux" and token.text.lower() == "when":
            return True
        elif token.dep_ == "prep" and token.text.lower() in ["during", "at"]:
            return True
        elif token.text.lower() in ["after", "before", "tomorrow", "yesterday"]:
            return True
        elif token.dep_ == "aux" and (token.text.lower() == "will" or token.text.lower() == "is" or token.text.lower() == "are"):
            return True
        elif token.dep_ == "ROOT" and (token.text.lower() == "is" or token.text.lower() == "are"):
            return True
    return False


data = pd.read_csv("/content/drive/MyDrive/Aru/question_Samsum_test.csv",nrows=200)
filtered_data = data[data.apply(lambda row: check_temporal_connotation(row["predicted_question"], row["context"]), axis=1)]

<ipython-input-5-670a040d0cc5>:17: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_scores = [phrase_doc.similarity(nlp(np)) for np in noun_phrases]
